In [108]:
import numpy as np
import pandas as pd
import os
from glob import glob
import optuna
from sklearn.metrics import mean_absolute_percentage_error

In [109]:
oof_root_path = '/home/nuwaisir/Corridor/Contests/Shell_ai_fuel_blend/Ensemble_tabm'

In [110]:
sorted(glob(os.path.join(os.path.join('./Ensemble_tabm', 'BlendProperty9'), "*.npy")))

['./Ensemble_tabm/BlendProperty9/oof_b9_seed_500_splits_10_score_87.60421484880118.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_600_splits_10_score_83.40528603839434.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_60_splits_10_score_87.37089214075809.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_700_splits_10_score_86.47618938076656.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_70_splits_10_score_87.88429789694496.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_800_splits_10_score_86.19291553143182.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_80_splits_10_score_88.33812320396292.npy',
 './Ensemble_tabm/BlendProperty9/oof_b9_seed_90_splits_10_score_86.45741681141442.npy']

In [111]:
import numpy as np
import pandas as pd
import os
from glob import glob
import optuna
from sklearn.metrics import mean_absolute_percentage_error

def load_oof_files(folder_path):
    oof_list = []
    
    for path in sorted(glob(os.path.join(folder_path, "*.npy"))):
        arr = np.load(path)
        oof_list.append(arr)
    return np.column_stack(oof_list)

def optimize_weights(oof_stack, y_true, n_trials=100):
    def objective(trial):
        weights = np.array([trial.suggest_float(f"w{i}", 0, 1) for i in range(oof_stack.shape[1])])
        weights /= weights.sum()
        pred = np.dot(oof_stack, weights)
        mape = mean_absolute_percentage_error(y_true, pred)  # SV metric
        return 100 - 90 * mape / 2.72

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    best_weights = np.array([study.best_params[f"w{i}"] for i in range(oof_stack.shape[1])])
    best_weights /= best_weights.sum()
    return best_weights

def ensemble_oof_predictions(train_csv, oof_root, target_cols, output_dir="ensemble_oof", n_trials=100):
    os.makedirs(output_dir, exist_ok=True)
    train_df = pd.read_csv(train_csv)
    results = {}

    for target in target_cols:
        print(f"\n▶ Ensembling {target}")
        y_true = train_df[target].values
        folder = os.path.join(oof_root, target)
        oof_stack = load_oof_files(folder)

        print(f"   - {oof_stack.shape[1]} models")
        weights = optimize_weights(oof_stack, y_true, n_trials)
        print(weights)
        final_pred = np.dot(oof_stack, weights)
        # corr = np.corrcoef(final_pred, y_true)[0, 1]
        mape = mean_absolute_percentage_error(y_true, final_pred)  # SV metric
        score = 100 - 90 * mape / 2.72

        np.save(os.path.join(output_dir, f"{target}_ensemble.npy"), final_pred)
        results[target] = {"Score": score, "weights": weights.tolist()}
        print(f"   ✅ Score: {score:.6f}")

    return results

# Example usage
if __name__ == "__main__":
    target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
    results = ensemble_oof_predictions(
        train_csv="./dataset/train.csv",
        oof_root="./Ensemble_tabm",  # or your new directory
        target_cols=target_cols,
        output_dir="./ensemble_oof",
        n_trials=100
    )

    print("\n📊 Final SV Scores:")
    for k, v in results.items():
        print(f"{k}: {v['Score']:.6f}")


[I 2025-07-23 18:24:21,399] A new study created in memory with name: no-name-e1bc314f-7e88-4987-ba29-00bdcdf66323
[I 2025-07-23 18:24:21,402] Trial 0 finished with value: 97.8054407311468 and parameters: {'w0': 0.7828484643585403, 'w1': 0.8154391872264497, 'w2': 0.7419260747833935, 'w3': 0.6038420262216808, 'w4': 0.9034505542604536, 'w5': 0.9866087954984653, 'w6': 0.828193801640999, 'w7': 0.9829101751775964}. Best is trial 0 with value: 97.8054407311468.
[I 2025-07-23 18:24:21,403] Trial 1 finished with value: 97.58795564606955 and parameters: {'w0': 0.11583337321830478, 'w1': 0.21271392513002285, 'w2': 0.9215931405745266, 'w3': 0.9814896514204338, 'w4': 0.6851428762654225, 'w5': 0.870356251853508, 'w6': 0.6928466539945582, 'w7': 0.05655691159418985}. Best is trial 0 with value: 97.8054407311468.
[I 2025-07-23 18:24:21,404] Trial 2 finished with value: 97.57442736423683 and parameters: {'w0': 0.08932636913153935, 'w1': 0.33272179803153923, 'w2': 0.32820407548261576, 'w3': 0.92357124084

[I 2025-07-23 18:24:21,416] Trial 6 finished with value: 97.91425795634208 and parameters: {'w0': 0.872091372245302, 'w1': 0.5151380277581946, 'w2': 0.28747762099571617, 'w3': 0.11384482684164221, 'w4': 0.3924638909848974, 'w5': 0.6999845168297572, 'w6': 0.18311754043190198, 'w7': 0.6850025839165742}. Best is trial 6 with value: 97.91425795634208.
[I 2025-07-23 18:24:21,417] Trial 7 finished with value: 97.58081521472702 and parameters: {'w0': 0.5994574468854822, 'w1': 0.07637854424420298, 'w2': 0.035031269243300356, 'w3': 0.9095149926551886, 'w4': 0.8682116659054657, 'w5': 0.039046736733118315, 'w6': 0.9196660406595989, 'w7': 0.4098265281317989}. Best is trial 6 with value: 97.91425795634208.
[I 2025-07-23 18:24:21,419] Trial 8 finished with value: 97.81274954908393 and parameters: {'w0': 0.6566257315925693, 'w1': 0.9766061131627753, 'w2': 0.09057926273401795, 'w3': 0.514800332668623, 'w4': 0.955415450084479, 'w5': 0.4986348450478765, 'w6': 0.6794459392390976, 'w7': 0.3144510018057231


▶ Ensembling BlendProperty1
   - 8 models


[I 2025-07-23 18:24:21,606] Trial 17 finished with value: 97.92868560299326 and parameters: {'w0': 0.9649843601636263, 'w1': 0.674943923157479, 'w2': 0.16279292309272778, 'w3': 0.7610698295684, 'w4': 0.3507173202715246, 'w5': 0.9093951118274908, 'w6': 0.511616737775657, 'w7': 0.8092566488746539}. Best is trial 11 with value: 97.98624599742082.
[I 2025-07-23 18:24:21,627] Trial 18 finished with value: 97.90024236088108 and parameters: {'w0': 0.796756639711743, 'w1': 0.8895945902159645, 'w2': 0.6160653818625303, 'w3': 0.3159880468921119, 'w4': 0.5591804767179431, 'w5': 0.6984567753242573, 'w6': 0.18977571332524318, 'w7': 0.48030696314066723}. Best is trial 11 with value: 97.98624599742082.
[I 2025-07-23 18:24:21,650] Trial 19 finished with value: 97.90327295876615 and parameters: {'w0': 0.3108213996231366, 'w1': 0.38634302383720465, 'w2': 0.14260594932434079, 'w3': 0.09826793526497324, 'w4': 0.1312341482047248, 'w5': 0.7071497307554948, 'w6': 0.0006256100391234254, 'w7': 0.87661129937306

[0.26785191 0.26524561 0.06268138 0.14698352 0.03132765 0.17776922
 0.00148419 0.04665651]
   ✅ Score: 98.050387

▶ Ensembling BlendProperty2
   - 8 models


...

[0.10466089 0.22729509 0.25555836 0.02361443 0.19522112 0.06388207
 0.09763543 0.03213262]
   ✅ Score: 92.990060

▶ Ensembling BlendProperty9
   - 8 models


[I 2025-07-23 18:24:43,617] Trial 16 finished with value: 89.23526260741268 and parameters: {'w0': 0.7048124996136194, 'w1': 0.14907401077541094, 'w2': 0.5925694643816889, 'w3': 0.6483425237682767, 'w4': 0.793915054957049, 'w5': 0.422169033728311, 'w6': 0.6933528922394827, 'w7': 0.3792084352799813}. Best is trial 14 with value: 89.28672274328049.
[I 2025-07-23 18:24:43,646] Trial 17 finished with value: 88.57530592350007 and parameters: {'w0': 0.7322573394093549, 'w1': 0.9763967454068623, 'w2': 0.29032106079739867, 'w3': 0.5820327421615792, 'w4': 0.6141829780016946, 'w5': 0.4205454335149994, 'w6': 0.6900580100015945, 'w7': 0.4825633112541904}. Best is trial 14 with value: 89.28672274328049.
[I 2025-07-23 18:24:43,675] Trial 18 finished with value: 89.17570635297984 and parameters: {'w0': 0.9878977490460096, 'w1': 0.16969957655600076, 'w2': 0.5695000326895975, 'w3': 0.8059831116531709, 'w4': 0.8187752078306325, 'w5': 0.4106463703408286, 'w6': 0.6823136878216851, 'w7': 0.7339714128698112

[0.24932223 0.00315602 0.15340647 0.14938109 0.18169854 0.00783649
 0.23433616 0.02086299]
   ✅ Score: 89.437829

▶ Ensembling BlendProperty10
   - 8 models


[I 2025-07-23 18:24:46,493] Trial 16 finished with value: 96.7486755299947 and parameters: {'w0': 0.010347437429833184, 'w1': 0.969196325308537, 'w2': 0.6201291314840759, 'w3': 0.5400158818210702, 'w4': 0.7878123602515901, 'w5': 0.8047666982369465, 'w6': 0.8272637030125722, 'w7': 0.42173969610792283}. Best is trial 16 with value: 96.7486755299947.
[I 2025-07-23 18:24:46,514] Trial 17 finished with value: 96.75954814476731 and parameters: {'w0': 0.19033090827420718, 'w1': 0.9704428199415792, 'w2': 0.32346321377791076, 'w3': 0.5172315829975658, 'w4': 0.7604926440201424, 'w5': 0.7863250037365284, 'w6': 0.8007068910050041, 'w7': 0.32002523060832916}. Best is trial 17 with value: 96.75954814476731.
[I 2025-07-23 18:24:46,536] Trial 18 finished with value: 96.73208412267438 and parameters: {'w0': 0.17509384914007925, 'w1': 0.9389875115870291, 'w2': 0.310130027319103, 'w3': 0.5007072053817556, 'w4': 0.7701145617800442, 'w5': 0.5416311268523488, 'w6': 0.806946505638662, 'w7': 0.311458988961443

[0.00113443 0.25254553 0.04636554 0.00666112 0.15478599 0.22837566
 0.24887391 0.06125783]
   ✅ Score: 96.963319

📊 Final SV Scores:
BlendProperty1: 98.050387
BlendProperty2: 98.310241
BlendProperty3: 89.401630
BlendProperty4: 98.192473
BlendProperty5: 99.034585
BlendProperty6: 98.188228
BlendProperty7: 91.289358
BlendProperty8: 92.990060
BlendProperty9: 89.437829
BlendProperty10: 96.963319


In [113]:
df_submission = pd.read_csv('./dataset/sample_solution.csv')

In [114]:
# for i in range(1, 10 + 1):
#     pred = np.load(f'./ensemble_oof/BlendProperty{i}_ensemble.npy')
#     print(len(pred))
#     break
# #     df_submission[f'BlendProperty{i}'] = pred
# # df_submission.to_csv('./submission_ensemble_seeds.csv')
    

In [115]:
import os
import numpy as np
import pandas as pd
from glob import glob

def group_test_predictions_by_target(folder_path):
    """
    Groups prediction file paths by target index.
    Returns: dict[target_index] = list of file paths
    """
    grouped = {}
    # print(sorted(glob(os.path.join(folder_path, "*.npy"))))
    for path in sorted(glob(os.path.join(folder_path, "*.npy"))):
        base = os.path.basename(path)
        if base.startswith("b") and "_fold_" in base:
            tid = int(base.split("_")[0][1:])  # 'b2' -> 2
            grouped.setdefault(tid, []).append(path)
    return grouped

def generate_submission_from_seeded_preds(pred_folder, results_dict, output_csv):
    target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
    grouped_paths = group_test_predictions_by_target(pred_folder)
    submission = pd.read_csv('./dataset/sample_solution.csv')
    

    for tid, target in enumerate(target_cols, start=1):
        print(f"\n▶ Ensembling test predictions for {target}")
        paths = grouped_paths.get(tid, [])
        preds = [np.load(p) for p in sorted(paths)]
        stack = np.column_stack(preds)

        weights = np.array(results_dict[target]['weights'])
        weights /= weights.sum()

        final_pred = np.dot(stack, weights)
        submission[target] = final_pred

    submission.to_csv(output_csv, index=False)
    print(f"\n✅ Final submission saved to: {output_csv}")


In [116]:
# results = ensemble_oof_predictions()

In [106]:
# You already have Optuna weights in `results` dict
# Use the same one from the earlier ensemble

generate_submission_from_seeded_preds(
    pred_folder="./pred_10_fold",
    results_dict=results,
    output_csv="./submission_seed_ensemble.csv"
)



▶ Ensembling test predictions for BlendProperty1

▶ Ensembling test predictions for BlendProperty2

▶ Ensembling test predictions for BlendProperty3

▶ Ensembling test predictions for BlendProperty4

▶ Ensembling test predictions for BlendProperty5

▶ Ensembling test predictions for BlendProperty6

▶ Ensembling test predictions for BlendProperty7

▶ Ensembling test predictions for BlendProperty8

▶ Ensembling test predictions for BlendProperty9

▶ Ensembling test predictions for BlendProperty10

✅ Final submission saved to: ./submission_seed_ensemble.csv


In [117]:
df_best = pd.read_csv('submission_cur_best+tabm_b1234_10fold(cb)_93.92695.csv')
df_out = pd.read_csv('./submission_seed_ensemble.csv')

for target_col in target_cols:
    if target_col not in ['BlendProperty3', 'BlendProperty7', 'BlendProperty9']:
        df_best[target_col] = df_out[target_col]
        
df_best.to_csv('final4.csv', index=False)

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# from glob import glob

# def group_test_predictions_by_target(folder_path):
#     """
#     Groups prediction file paths by target index.
#     Returns: dict[target_index] = list of file paths
#     """
#     grouped = {}
#     print(sorted(glob(os.path.join(folder_path, "*.npy"))))
#     for path in sorted(glob(os.path.join(folder_path, "*.npy"))):
#         base = os.path.basename(path)
#         if base.startswith("b") and "_fold_" in base:
#             tid = int(base.split("_")[0][1:])  # 'b2' -> 2
#             grouped.setdefault(tid, []).append(path)
#     return grouped

# def generate_submission_from_seeded_preds(pred_folder, results_dict_root, output_csv):
#     target_cols = [f"BlendProperty{i}" for i in range(1, 11)]
#     grouped_paths = group_test_predictions_by_target(pred_folder)
#     submission = pd.read_csv("./dataset/sample_solution.csv")

#     for tid, target in enumerate(target_cols, start=1):
#         print(f"\n▶ Ensembling test predictions for {target}")
#         paths = grouped_paths.get(tid, [])

#         # Sort paths by their embedded score in descending order (optional filtering)
#         paths = sorted(paths, key=lambda x: float(x.split("_score_")[-1].replace(".npy", "")), reverse=True)

#         preds = [np.load(p) for p in sorted(paths)]
#         stack = np.column_stack(preds)

#         # Load Optuna weights for this target
#         results_path = os.path.join(results_dict_root, target, "optuna_weights.npy")
#         weights = np.load(results_path)

#         weights = weights[:stack.shape[1]]  # Ensure shape match in case of filtering
#         weights /= weights.sum()

#         final_pred = np.dot(stack, weights)
#         submission[target] = final_pred

#     submission.to_csv(output_csv, index=False)
#     print(f"\n✅ Final submission saved to: {output_csv}")

# # Example call:
# generate_submission_from_seeded_preds(
#     pred_folder="./pred_10_fold",
#     results_dict_root="./Ensemble_tabm",
#     output_csv="./submission_seed_ensemble.csv"
# )


In [ ]:
df_1 = pd.read_csv("/home/nuwaisir/Corridor/Contests/Shell_ai_fuel_blend/Submissions/submission_autogluon_avg_7_86.29786.csv")
df_2 = pd.read_csv("/home/nuwaisir/Corridor/Contests/Shell_ai_fuel_blend/Submissions/submission_autogluon_avg_86.45445+85.35792.csv")
df_3 = pd.read_csv("/home/nuwaisir/Corridor/Contests/Shell_ai_fuel_blend/Submissions/submission_autogluon_avg_all.csv")
df_4 = pd.read_csv("./Submissions/submission_autogluon_exp_quality_85.29565.csv")
df_5 = pd.read_csv("./Submissions/submission_autogluon_exp_quality_85.42446.csv")